In [1]:
!pip install fsspec==2024.10.0

In [2]:
!pip install ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 26.1 MB/s eta 0:00:00
   ━━━━

## Prepare Data

In [3]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved this notebook
# e.g. 'CSE156/assignments/PA3/'
FOLDERNAME = "CSE_256_PA3"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This is later used to use the IMDB reviews
%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/CSE_256_PA3


In [4]:
import csv
import sys

def set_csv_field_limit():
    maxInt = sys.maxsize
    while True:
        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)
    return maxInt

def load_documents(doc_file):
    """
    Loads the document contents from the first file.

    :param doc_file: Path to the document file (document ID <TAB> document contents).
    :return: A dictionary {document_id: document_contents}.
    """
    # Set the field size limit first
    set_csv_field_limit()

    documents = {}
    with open(doc_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row)==0: continue
            doc_id, content = row
            documents[doc_id] = content
    return documents

docs = []
doc_file = 'meetings.tsv'
documents = load_documents(doc_file)
documents['doc_0']

"project manager: yep . soon as i get this . okay . this is our last meeting . um i 'll go ahead and go through the minutes from the previous meeting . uh and then we 'll have a , the prototype presentation . um then we will um do an evaluation . uh or we 'll see what , what we need to have under the criteria for the evaluation . then we 'll go through the finance and see if we fall within the budget . um then we 'll do the evaluation , and then we can finish up after that with um any changes that we 'll need to make , or hopefully everything will fall right in line . um let 's see , minutes from the last meeting . um we looked at uh the the trends . we had uh the fashion trends that people want a fancy look-and-feel . it was twice as important as anything else . um they liked fruit and vegetables in the new styles . um and a spongy feel . so we were talking about trying to incorporate those into our prototype . um they wanted limited buttons and simplicity . um then we looked at the u

In [33]:
import random

def load_questions_answers(qa_file):
    """
    Loads the questions and corresponding ground truth document IDs.

    :param qa_file: Path to the question-answer file (document ID <TAB> question <TAB> answer).
    :return: A list of tuples [(document_id, question, answer)].
    """
    qa_pairs = []
    with open(qa_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            doc_id, question, answer = row
            qa_pairs.append((doc_id, question, answer))

    random.shuffle(qa_pairs)

    return qa_pairs

qa_file = 'questions_answers.tsv'  # document ID <TAB> question <TAB> answer
qa_pairs = load_questions_answers(qa_file)
qa_pairs[0]


('doc_94',
 'why did project manager reject custom-designed chips initially when discussing components design ?',
 'project manager learned that custom-designed chips would be expensive and the completion time could not be predicted as the voice technology had not been well developed yet .')

In [6]:
my_collections = [documents[k] for k in documents.keys()]
doc_ids = list(documents.keys())
queries = [q for _, q, _ in qa_pairs]

## Load Model

### Load from Pretrained

In [7]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
index_path = RAG.index(index_name="my_index", collection=my_collections, document_ids=doc_ids, max_document_length=512, split_documents=True)

In [8]:
RAG = RAGPretrainedModel.from_index("./.ragatouille/colbert/indexes/my_index")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


### Fine-tune

In [20]:
from ragatouille import RAGTrainer
trainer = RAGTrainer(model_name="my_trained_index", pretrained_model_name="colbert-ir/colbertv2.0", language_code="en")

In [21]:
pairs = [(question, answer, doc_id) for doc_id, question, answer in qa_pairs]

In [23]:
trainer.prepare_training_data(raw_data=pairs, data_out_path="data_out", all_documents=documents, num_new_negatives=10, mine_hard_negatives=True)

Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...
Building hard negative index for 1380 documents...
All documents embedded, now adding to index...
save_index set to False, skipping saving hard negative index
Hard negative index generated


'data_out'

In [24]:
trainer.train(batch_size=32,
              nbits=4, # How many bits will the trained model use when compressing indexes
              maxsteps=500000, # Maximum steps hard stop
              use_ib_negatives=True, # Use in-batch negative to calculate loss
              dim=128, # How many dimensions per embedding. 128 is the default and works well.
              learning_rate=5e-6, # Learning rate, small values ([3e-6,3e-5] work best if the base model is BERT-like, 5e-6 is often the sweet spot)
              doc_maxlen=256, # Maximum document length. Because of how ColBERT works, smaller chunks (128-256) work very well.
              use_relu=False, # Disable ReLU -- doesn't improve performance
              warmup_steps="auto", # Defaults to 10%
             )

#> Starting...
#> Joined...


In [36]:
RAG = RAGPretrainedModel.from_pretrained(".ragatouille/colbert/checkpoints/colbert")
index_path = RAG.index(index_name="trained_index", collection=my_collections, document_ids=doc_ids, max_document_length=512, split_documents=True)

INFO: Tried to clean up context but failed!
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Dec 05, 16:12:54] #> Creating directory .ragatouille/colbert/indexes/trained_index 


[Dec 05, 16:12:55] [0] 		 #> Encoding 6493 passages..


/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


[Dec 05, 16:13:47] [0] 		 avg_doclen_est = 374.7378845214844 	 len(local_sample) = 6,493
[Dec 05, 16:13:49] [0] 		 Creating 16,384 partitions.
[Dec 05, 16:13:49] [0] 		 *Estimated* 2,433,173 embeddings.
[Dec 05, 16:13:49] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/trained_index/plan.json ..


/usr/local/lib/python3.10/dist-packages/colbert/indexing/collection_indexer.py:256: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_sample = torch.load(sub_sample_path)
/u

used 20 iterations (2.7248s) to cluster 2383173 items into 16384 clusters
[0.032, 0.034, 0.034, 0.03, 0.031, 0.034, 0.031, 0.032, 0.031, 0.033, 0.032, 0.033, 0.032, 0.034, 0.033, 0.036, 0.032, 0.033, 0.033, 0.031, 0.033, 0.035, 0.031, 0.031, 0.031, 0.031, 0.035, 0.032, 0.032, 0.033, 0.034, 0.034, 0.035, 0.032, 0.031, 0.03, 0.031, 0.032, 0.033, 0.039, 0.036, 0.031, 0.033, 0.034, 0.032, 0.033, 0.03, 0.035, 0.033, 0.032, 0.033, 0.031, 0.036, 0.031, 0.03, 0.033, 0.034, 0.035, 0.037, 0.031, 0.033, 0.03, 0.034, 0.035, 0.034, 0.037, 0.034, 0.031, 0.032, 0.033, 0.033, 0.027, 0.032, 0.035, 0.034, 0.031, 0.035, 0.032, 0.035, 0.035, 0.035, 0.033, 0.032, 0.033, 0.03, 0.032, 0.036, 0.032, 0.032, 0.038, 0.036, 0.034, 0.031, 0.033, 0.032, 0.035, 0.035, 0.03, 0.033, 0.032, 0.033, 0.036, 0.031, 0.035, 0.034, 0.033, 0.032, 0.033, 0.032, 0.033, 0.032, 0.032, 0.033, 0.034, 0.031, 0.031, 0.035, 0.033, 0.031, 0.032, 0.03, 0.033, 0.032, 0.033, 0.031, 0.034, 0.03, 0.036]


...

1it [00:53, 53.77s/it]
  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

[Dec 05, 16:14:50] #> Optimizing IVF to store map from centroids to list of pids..
[Dec 05, 16:14:50] #> Building the emb2pid mapping..


[Dec 05, 16:14:50] len(emb2pid) = 2433173


100%|██████████| 16384/16384 [00:00<00:00, 61548.74it/s]


[Dec 05, 16:14:50] #> Saved optimized IVF to .ragatouille/colbert/indexes/trained_index/ivf.pid.pt
Done indexing!


## Evaluate

In [9]:
query = 'what did kirsty williams am say about her plan for quality assurance ?'
results = RAG.search(query, k=5)

Loading searcher for index my_index for the first time... This may take a few seconds
[Dec 05, 15:34:41] #> Loading codec...


/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids = torch.load(centroids_path, map_loc

[Dec 05, 15:34:42] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Dec 05, 15:36:15] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


...

100%|██████████| 1/1 [00:00<00:00,  2.66it/s]

[Dec 05, 15:37:48] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . what did kirsty williams am say about her plan for quality assurance ?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2106, 11382, 12096,  2100,  3766,  2572,  2360,
         2055,  2014,  2933,  2005,  3737, 16375,  1029,   102,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


In [11]:
retrieved_docs = [i['document_id'] for i in results]
retrieved_docs

['doc_2', 'doc_2', 'doc_176', 'doc_211', 'doc_211']

In [29]:
import time
def precision_at_k(ground_truth, retrieved_docs, k):
    """
    Computes Precision at k for a single query.

    :param ground_truth: The name of the ground truth document.
    :param retrieved_docs: The list of document names returned by the model in ranked order.
    :param k: The cutoff for computing Precision.
    :return: Precision at k.
    """
    return 1 if ground_truth in retrieved_docs[:k] else 0

def evaluate(model, qa_pairs, ranking_function=None, max_k=5):
    """
    Evaluate the retrieval system based on the documents and question-answer pairs.
    Calculates precision@k for k from 1 to max_k.

    :param qa_pairs: List of (doc_id, question, answer) tuples
    :param ranking_function: Optional ranking function
    :param max_k: Maximum k value for precision@k calculation
    """
    # Initialize precision scores for each k
    precision_scores = {k: [] for k in range(1, max_k + 1)}

    for doc_id, question, _ in qa_pairs:
        # Get enough results for maximum k
        results = model.search(question, k=max_k)
        retrieved_docs = [i['document_id'] for i in results]

        # Calculate precision for each k
        for k in range(1, max_k + 1):
            precision_scores[k].append(precision_at_k(doc_id, retrieved_docs, k))

        # Print progress every 10 queries
        if len(precision_scores[1]) % 10 == 0:
            print(f"\nAfter {len(precision_scores[1])} queries:")
            for k in range(1, max_k + 1):
                avg_precision = sum(precision_scores[k]) / len(precision_scores[k])
                print(f"Precision@{k}: {avg_precision:.3f}")

    # Print final results
    print("\nFinal Results:")
    for k in range(1, max_k + 1):
        avg_precision = sum(precision_scores[k]) / len(precision_scores[k])
        print(f"Precision@{k}: {avg_precision:.3f}")

start_time = time.time()
evaluate(RAG, qa_pairs, max_k=5)
end_time = time.time()
elapsed_time = (end_time - start_time)/60
print(f"\nTime taken: {elapsed_time:.2f} minutes")